In [17]:
from astropy.io import fits


image = fits.open('solved_calibrated_files/calib_red_20s.000.new')

hdr = image[0].header
print(hdr)

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                 1024                                                  NAXIS2  =                 1024                                                  EXTEND  =                    T                                                  COMMENT Original key: "END"                                                     COMMENT                                                                         COMMENT --Start of Astrometry.net WCS solution--                                COMMENT --Put in by the new-wcs program--                                       COMMENT                                                                         WCSAXES =                    2 / no comment                                     CTYPE1  = 'RA---TAN-SIP' / TAN (gnomic) 

In [16]:
import os
import numpy as np
from astropy.io import fits

# Get a list of all files in the directory
file_list = os.listdir('sextractor')

# Filter and sort files numerically based on the number part before the extension
file_list = [file_name for file_name in file_list if file_name.startswith('calib_red_20s') and file_name.endswith('.cat')]
file_list.sort(key=lambda x: int(x.split('.')[-3].split('_')[-1]))

# Open the data file for writing
with open('data.dat', 'w') as f:
    # Iterate over each file in the directory
    for file_name in file_list:
        # Extract file number
        file_number = file_name.split('.')[-3].split('_')[-1]

        # Extract time of observation from FITS header
        fits_file_path = os.path.join('solved_calibrated_files', 'calib_red_20s.' + file_number + '.new')
        with fits.open(fits_file_path) as hdul:
            header = hdul[0].header
            time_of_observation = header['BITPIX']

        # Load data from file
        data = np.loadtxt(os.path.join('sextractor', file_name))

        # Extract columns from the data
        x_pix = data[:, 1]
        right_ascensions = data[:, 3]
        decs = data[:, 4]
        flux = data[:, 5]
        flux_err = data[:, 6]

        # Define masks for right ascensions and declinations
        ra_mask = (right_ascensions < 117.0285) & (right_ascensions > 117.0266)
        dec_mask = (decs < 50.227) & (decs > 50.223)

        # Combine masks using logical AND
        combined_mask = ra_mask & dec_mask

        # Apply the combined mask to get the indices where the condition is True
        indices = np.where(combined_mask)

        # Extract the values based on the combined mask
        xo_flux = flux[indices]
        xo_flux_err = flux_err[indices]

        # Stack flux and flux error horizontally
        stacked_data = np.column_stack((xo_flux, xo_flux_err))

        # Write the file name, time of observation, and the stacked data to the data file
        f.write(f"{time_of_observation}\n")
        np.savetxt(f, stacked_data)
        f.write('\n')
